In [1]:
import os
from libv3.utils import *
import pandas as pd
import json

In [2]:
############ configuration ################
############################################

CODE, BEHAVIOUR, THREAD, VER = get_config()   ### config stored in libv3/exp_config.txt

base_dir = '../trace_data' ### can be replaced with 'csv', 'exe_plot', 'histogram'
log_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR}'

print(log_path)

#### file to display
trace_file = 0

print('file number:', trace_file)

CODE: theft_protection
BEHAVIOUR: faulty_data
THREAD: single
VER: 3
../trace_data/theft_protection/single_thread/version_3/faulty_data
file number: 0


In [3]:
######### get paths #######################
paths_log, paths_traces, varlist_path, paths_label = get_paths(log_path)

paths_log.sort()
paths_traces.sort()
varlist_path.sort()

print(paths_log)
print(paths_traces)
print(varlist_path)
print(paths_label)

['../trace_data/theft_protection/single_thread/version_3/faulty_data/log1-com', '../trace_data/theft_protection/single_thread/version_3/faulty_data/log2-sensor']
['../trace_data/theft_protection/single_thread/version_3/faulty_data/trace1-com', '../trace_data/theft_protection/single_thread/version_3/faulty_data/trace2-sensor']
['../trace_data/theft_protection/single_thread/version_3/faulty_data/varlist1-com', '../trace_data/theft_protection/single_thread/version_3/faulty_data/varlist2-sensor']
['../trace_data/theft_protection/single_thread/version_3/faulty_data/labels/trace1-com_label.json', '../trace_data/theft_protection/single_thread/version_3/faulty_data/labels/trace2-sensor_label.json']


In [4]:
############# check varlist is consistent ############
############# only for version 3 ######################

if VER == 3:
    to_number = is_consistent(varlist_path)

    if to_number != False:
        from_number = mapint2var(to_number)


varlist 1 is consistent with varlist 0
{'1_0_main_g_ack': 13, '1_control_update_txmsg_0': 12, '1_0_main_lora': 2, '1_0_main_temperature': 8, '1_0_main_ow': 0, '1_0_main_events': 14, '1_0_main_com_timer': 4, '1_control_update_rxmsg_drop': 15, '1_0_loracom_data': 11, '1_control_readdata_0': 10, '1_0_main_temp': 1, '1_0_main_s': 3, '1_control_updatedata_cls.sensor_data': 9, '1_0_sense_temperature': 7, '1_0_main_i': 6, '1_control_init_timer0_0': 5} {'1_0_main_g_ack': 13, '1_control_update_txmsg_0': 12, '1_0_main_lora': 2, '1_0_main_temperature': 8, '1_0_main_ow': 0, '1_0_main_events': 14, '1_0_main_com_timer': 4, '1_control_update_rxmsg_drop': 15, '1_0_loracom_data': 11, '1_control_readdata_0': 10, '1_0_main_temp': 1, '1_0_main_s': 3, '1_control_updatedata_cls.sensor_data': 9, '1_0_sense_temperature': 7, '1_0_main_i': 6, '1_control_init_timer0_0': 5}


In [5]:
############ Get variable list ######################
sorted_keys = list(from_number.keys())
sorted_keys.sort()
var_list = [from_number[key] for key in sorted_keys]   ### get the variable list
# print(var_list)

## Trace Plots

### Process Traces

In [6]:
########## process the traces ###########
col_data = preprocess_traces(paths_traces, var_list)   ### in the format (trace_name, x_data, y_data, y_labels, trace_path) 

../trace_data/theft_protection/single_thread/version_3/faulty_data/trace1-com trace1-com
../trace_data/theft_protection/single_thread/version_3/faulty_data/trace2-sensor trace2-sensor


### Generate plot trace data

In [7]:
############### preprocessing data to plot using plotly ##############
'''
Restructure the data in dictionary with (keys,value) pair :-  (time, timestamps) , (trace_name, trace)
'''
all_df = get_dataframe(col_data)   

In [8]:
all_df[0]

,time,trace1-com
0,1307,0
1,1335,1
2,1356,2
3,1361,3
4,1869,4
...,...,...
2566,532119,8
2567,532124,9
2568,532127,10
2569,532133,11


In [ ]:

### plot single trace
for i, df in enumerate(all_df):
    if i == trace_file:
        plot_single_trace(df, var_list, with_time=False, is_xticks=True)

In [ ]:
### get timestamp

timestamp = index2timestamp(all_df[0], 227)
print(timestamp)

## Execution Interval Plots

### Preprocess data

In [ ]:
####### preprocessing data ########

#### extract timestamps for each variable and store them in a dictionary along with index values for each variable in event trace

var_timestamps = get_var_timestamps(paths_traces)    #### in format (filename, dict of timestamps and index values)

to_plot = preprocess_variable_plotting(var_timestamps, var_list, from_number, trace_number=trace_file)   ### restructure the data for plotting


### Generate execution interval plots

In [ ]:
'''
TODO:
- Add option to plot anomalies in trace plotting
- Add option to plot variable exe interval for single trace (added)
- Define format of detected anomalies for plotting (done)
'''

In [ ]:
### plot the data
plot_execution_interval_single(to_plot, is_xticks=False)

## Plot Labels

In [9]:
### read labels
'''
labels are of format [index1, index2, timestamp1, timestamp2, class]
'''
for i, path in enumerate(paths_label):
    if i == trace_file:
        print(path)
        label_content = prepare_gt(path)

../trace_data/theft_protection/single_thread/version_3/faulty_data/labels/trace1-com_label.json


In [11]:
var_list

['1_0_main_g_ack',
 '1_control_update_txmsg_0',
 '1_0_main_lora',
 '1_0_main_temperature',
 '1_0_main_ow',
 '1_0_main_events',
 '1_0_main_com_timer',
 '1_control_update_rxmsg_drop',
 '1_0_loracom_data',
 '1_control_readdata_0',
 '1_0_main_temp',
 '1_0_main_s',
 '1_control_updatedata_cls.sensor_data',
 '1_0_sense_temperature',
 '1_0_main_i',
 '1_control_init_timer0_0']

In [10]:
### plot
for i, df in enumerate(all_df):
    if i == trace_file:
        plot_single_trace(df, var_list, with_time=False, is_xticks=True, ground_truths=label_content)

## Generate train data

In [ ]:
'''
Add the indices for the interval in 'normal_seq_inter' that show correct behvaiour in the traces. 
The format is as follows:
path_traces = [path1, path2, ...]
normal_seq_inter = [ ( intervals for trace1, ...), 
                    ( intervals for trace2 ...),
                      ...] 
'''

normal_seq_inter = (  ( (0,110), (150,900), (1250,2000), (2050,2300) ),  ### v3, normal trace0
            )

for p, n_inter in zip(paths_traces, normal_seq_inter):
    trace = read_traces(p)
    train_data_path = os.path.join(os.path.dirname(p), 'train_data')

    if not os.path.exists(train_data_path):
        os.makedirs(train_data_path)

    for i, inter in enumerate(n_inter):
        start, end = inter
        # trace[start:end].to_csv(os.path.join(train_data_path, f'interval_{start}_{end}.csv'), index=False)
        json.dump(trace[start:end], open(os.path.join(train_data_path, f'interval_{start}_{end}.json'), 'w'))

In [ ]:
normal_seq_inter[0]